In [9]:
Text='''
1. INTRODUCTION
This site is owned and operated by Peugeot Motor Company PLC (referred to as "Peugeot/we/our/us"), a public limited company registered in England under company registration number 148545, the registered office address of which is:

Pinley House
2 Sunbeam Way
Coventry
West Midlands
CV3 1ND
 

Peugeot Motor Company PLC is authorised and regulated by the Financial Conduct Authority.

As a user of our website (referred to as "you/your") you acknowledge that such use is subject to our terms and conditions set out below.

2. INTELLECTUAL PROPERTY
All intellectual property rights in connection with all the information and images on the website are the property of Peugeot Motor Company PLC. No information or images are to be used or duplicated without our prior written permission.
3. ACCESSIBILITY
All intellectual property rights in connection with all the information and images on the website are the property of Peugeot Motor Company PLC. No information or images are to be used or duplicated without our prior written permission.
4. PRICING AND GOVERNMENT VEHICLE EXCISE DUTY INFORMATION
The 'Manufacturer's Recommended Retail Price/On the Road price/OTR' featured on this website includes the following:

1)    Delivery to dealer and number plates:

Cars - £550.83 (+ VAT £110.17)

2)    Government Vehicle Excise Duty Bands (new first year rates effective from April 2014):

Band A (Up to 100 g/km) - £0
Band B (101-110) - £0
Band C (111-120) - £0
Band D (121-130) - £0
Band E (131-140) - £130
Band F (141-150) - £145
Band G (151-165) - £180
Band H (166-175) - £290
Band I (176-185) - £345
Band J (186-200) - £485
Band K (201-225) - £635
Band L (226-255) - £860
Band M (Over 255) - £1,090

3)    First Registration Fee - £55.00

All information and prices correct at time of publication, but are subject to change at any time without prior notice (which may occur as a result of matters including, but not limited to, any changes in legislation and/or any changes by the government).  Please see our Prices and Specs brochures or contact your local Peugeot dealer  for the most up-to-date prices and specification details for all models.
5. MODEL INFORMATION
The models shown in this website are for guidance only. Some features appear on Special Edition models only; others are optional at extra cost. For detailed specification of all vehicles in the Peugeot range, accessories, after sales information, prices and finance plans, please consult your local authorised Peugeot dealer.

It is our policy to continually improve our vehicles in every way and we reserve the right to modify our vehicles' specifications (including colour), equipment, accessories and characteristics or to discontinue any model, without notice, at any time. Every endeavour has been made to ensure that the information, details and descriptions set out in this website are correct. However, we are unable to guarantee that no changes have recently taken place to the specifications (including colour), equipment, accessories and characteristics of, or relating to, any Peugeot vehicle. Although every effort is made to bring this website up to date, from time to time, in order to avoid any misunderstandings, you are advised to check with your Peugeot dealer the specifications (including colour), equipment and accessories relating to any Peugeot vehicle, prior to ordering.
6. OFFICIAL GOVERNMENT FUEL CONSUMPTION FIGURES
Fuel consumption figures (Urban Drive Cycle MPG, Extra Urban Drive Cycle MPG, Combined Drive Cycle MPG and CO2 Emissions g/km) quoted on our website are achieved under official EU test conditions and are intended as a guide for comparative purposes only. Figures may not be reflected in actual on-the-road driving conditions.

'''

Text=Text.replace("\n"," ")

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
import os
import gc

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from langchain.text_splitter import CharacterTextSplitter

device = "cuda" # use cpu if needed
print("start_model")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/Mistral-7B-OpenOrca").to(device)
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")


start_model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def find_dimension(inputText):
    prompt = "I give you a text. You must find the address (if there is any. You must only write the address like <ADDR: address>. The text  is as follows: \n\n"+inputText+" \n &&&&&&&&&&&&&&&&&&& \n"
    prompt=prompt.replace("  "," ")
    prompt=prompt.lstrip().rstrip().strip()
    inputs = tokenizer(
        prompt,
        return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs, max_new_tokens=8000, use_cache=True, do_sample=True,
        temperature=0.1, top_p=0.95)

    bbb=tokenizer.batch_decode(outputs)[0].replace(inputText,"").lstrip().rstrip().strip().split("&&&&&&&&&&&&&&&&&&&")[-1]
    bbb=bbb.rstrip().lstrip().strip().split(": ")[-1].replace("<|im_end|>","").replace(">","").replace("<","").rstrip().lstrip().strip()
    return bbb 

print(find_dimension(Text))


Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Pinley House 2 Sunbeam Way Coventry West Midlands CV3 1ND
